In [1]:
import pandas as pd
import numpy as np
import datetime

## 工序A

In [2]:
df = pd.read_excel('附件一.xlsx')
def transform_time(time):
    hour = time.hour
    minute = time.minute
    return hour * 60 + minute
df['数值型时间'] = df['到站时间'].apply(transform_time)

time_car_dict = {}
for time, car in zip(df['数值型时间'], df['动车类别']):
    time_car_dict[time] = car
    
#time_car_dict = {0: 'CRH2',
#                31: 'CRH5',
#                66: 'CRH2',
#                104: 'CRH6',
#                125: 'CRH3',
#                166: 'CRH6',
#                195: 'CRH2',
#                223: 'CRH5',
#                225: 'CRH3',
#                251: 'CRH3',
#                293: 'CRH6'}
    
car_cost_A = {'CRH2': 60, "CRH3": 48 , "CRH5": 78 , "CRH6" : 60}

is_finish_A = np.zeros(shape = (len(df) ), dtype = int) # 记录所有动车的A工艺完成状态，1为已完成，0为未完成

m = len(df)

In [3]:
is_work_A = np.zeros(shape = (24*60 , 3), dtype = int)  ## 记录车间 [a1 ; a2 ; a3] 三个车间每分钟的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序A的时间
finish_time_record = []  ## 记录每个动车完成工序A的时间
car_type_record = []     ## 按顺序记录每次检修动车类别
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间

car_id = 0

for minute in range(len(is_work_A)):
    num_work_A = is_work_A[minute].sum()  ## 当前时间车间A的运作数量
    num_finish_A = is_finish_A.sum() ## 已经发出的机车数量
    if num_work_A == 0 and num_finish_A == m: ## 工艺完成
        break
    else:
        if num_work_A < 3 and minute in time_car_dict :## 有空闲车间 且 有新车进
            car_type = time_car_dict[minute]
            cost_time = car_cost_A[car_type]
            free_id = np.where(is_work_A[minute] == 0)[0][0]
            is_work_A[minute: minute + cost_time, free_id] = 1
            is_finish_A[car_id] = 1
            car_id += 1
            #print(car_id, minute + 1, car_type,free_id, cost_time )
            finish_time_record.append(minute + cost_time)
            car_type_record.append(car_type)
            input_time_record.append(minute)
            workshop_ind_record.append(free_id)
        
        if num_work_A == 3 and minute in time_car_dict :## 无空闲车间 且 有新车进，则推迟新车进来
            car_type = time_car_dict[minute]
            
            for k in range(10000):
                if minute + k not in time_car_dict:
                    time_car_dict[minute + k] = car_type
                    break
            
            
df_A = pd.DataFrame({'动车类别':car_type_record, 
                  '进站时间': input_time_record,
                  '出站时间': finish_time_record,
                  '检测车间编号':workshop_ind_record})

df_A = df_A[['动车类别','进站时间', '出站时间','检测车间编号']]

df_A['检测耗时(分钟)'] = df_A['出站时间'] - df_A['进站时间']

df_A['检测车间编号(展示)'] = df_A['检测车间编号'] + 1
#df_A.to_csv('render.csv', encoding = 'gbk' , index = None)   

## 工序B

In [4]:
## 工序A的出战时间为工序B的进站时间
df = df_A[['动车类别', '出站时间']]
df.columns = ['动车类别','工序b进站时间']


In [5]:
time_car_dict = {}
for time, car in zip(df['工序b进站时间'], df['动车类别']):
    time_car_dict[time] = car
    
car_cost_B = {'CRH2': 120, "CRH3": 144 , "CRH5": 150 , "CRH6" : 162}

is_finish_B = np.zeros(shape = (len(df) ), dtype = int) # 记录所有动车的B工艺完成状态，1为已完成，0为未完成

m = len(df)

In [6]:
is_work_B = np.zeros(shape = (24*60 , 8 ), dtype = int)  ## 记录车间 [b1,b2,...,b8] 8个车间每分钟的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序B的时间
finish_time_record = []  ## 记录每个动车完成工序B的时间
car_type_record = []     ## 按顺序记录每次检修动车类别
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间

car_id = 0

for minute in range(len(is_work_B)):
    num_work_B = is_work_B[minute].sum()  ## 当前时间车间B的运作数量
    num_finish_B = is_finish_B.sum()      ## 已经发出的机车数量
    if num_work_B == 0 and num_finish_B == m: ## 工艺完成, 则停止
        break
    else:
        if num_work_B < 8 and minute in time_car_dict :## 有空闲车间 且 有新车进
            car_type = time_car_dict[minute]
            cost_time = car_cost_B[car_type]
            free_id = np.where(is_work_B[minute] == 0)[0][0]
            is_work_B[minute: minute + cost_time, free_id] = 1
            is_finish_B[car_id] = 1
            car_id += 1
            #print(car_id, minute + 1, car_type,free_id, cost_time )
            finish_time_record.append(minute + cost_time)
            car_type_record.append(car_type)
            input_time_record.append(minute)
            workshop_ind_record.append(free_id)
        
        if num_work_B == 8 and minute in time_car_dict :## 无空闲车间 且 有新车进，则推迟新车进来
            car_type = time_car_dict[minute]
            
            for k in range(10000):
                if minute + k not in time_car_dict:
                    time_car_dict[minute + k] = car_type
                    break
            

In [7]:
df_B = pd.DataFrame({'动车类别':car_type_record, 
                  '进站时间': input_time_record,
                  '出站时间': finish_time_record,
                  '检测车间编号':workshop_ind_record})

df_B = df_B[['动车类别','进站时间', '出站时间','检测车间编号']]

df_B['检测耗时(分钟)'] = df_B['出站时间'] - df_B['进站时间']

df_B['检测车间编号(展示)'] = df_B['检测车间编号'] + 1
df_B.to_csv('render.csv', encoding = 'gbk' , index = None)

## 工序C

In [8]:
## 工序B的出战时间为工序C的进站时间
df = df_B[['动车类别', '出站时间']]
df.columns = ['动车类别','工序c进站时间']


In [9]:
time_car_dict = {}
for time, car in zip(df['工序c进站时间'], df['动车类别']):
    time_car_dict[time] = car
    
car_cost_C = {'CRH2': 90, "CRH3": 30 , "CRH5": 90 , "CRH6" : 18}

is_finish_C = np.zeros(shape = (len(df) ), dtype = int) # 记录所有动车的C工艺完成状态，1为已完成，0为未完成

m = len(df)

In [10]:
is_work_C = np.zeros(shape = (24*60 , 5 ), dtype = int)  ## 记录车间 [b1,b2,...,b8] 8个车间每分钟的工作状态。1为工作，0为空闲

input_time_record = []   ## 记录每个动车开始工序B的时间
finish_time_record = []  ## 记录每个动车完成工序B的时间
car_type_record = []     ## 按顺序记录每次检修动车类别
workshop_ind_record = [] ## 按顺序记录每次检修动车的车间

car_id = 0

for minute in range(len(is_work_C)):
    num_work_C = is_work_C[minute].sum()  ## 当前时间车间B的运作数量
    num_finish_C = is_finish_C.sum()      ## 已经发出的机车数量
    if num_work_C == 0 and num_finish_C == m: ## 工艺完成, 则停止
        break
    else:
        if num_work_C < 5 and minute in time_car_dict :## 有空闲车间 且 有新车进
            car_type = time_car_dict[minute]
            cost_time = car_cost_C[car_type]
            free_id = np.where(is_work_C[minute] == 0)[0][0]
            is_work_C[minute: minute + cost_time, free_id] = 1
            is_finish_C[car_id] = 1
            car_id += 1
            #print(car_id, minute + 1, car_type,free_id, cost_time )
            finish_time_record.append(minute + cost_time)
            car_type_record.append(car_type)
            input_time_record.append(minute)
            workshop_ind_record.append(free_id)
        
        if num_work_C == 5 and minute in time_car_dict :## 无空闲车间 且 有新车进，则推迟新车进来
            car_type = time_car_dict[minute]
            
            for k in range(10000):
                if minute + k not in time_car_dict:
                    time_car_dict[minute + k] = car_type
                    break
            

In [11]:
df_C = pd.DataFrame({'动车类别':car_type_record, 
                  '进站时间': input_time_record,
                  '出站时间': finish_time_record,
                  '检测车间编号':workshop_ind_record})

df_C = df_C[['动车类别','进站时间', '出站时间','检测车间编号']]

df_C['检测耗时(分钟)'] = df_C['出站时间'] - df_C['进站时间']

df_C['检测车间编号(展示)'] = df_C['检测车间编号'] + 1
df_C.to_csv('render.csv', encoding = 'gbk' , index = None)

### 计算总耗时

In [12]:
(df_C['出站时间'].max() - df_A['进站时间'].min() ) / 60 ## 小时

9.016666666666667